# Twitter

<center><img src="png/twitter_logo.png" width = 150/></center>

The third API we are going to explore is Twitter API. On the one hand, it will be the easiest one to access, on the other hand, it will be the hardest one because unlike Wikipedia and Reddit it requires having not only a Twitter account but also a Developer Account. Moreover, the version we are going to use has very strict limits on how much data we can actually get. However, enough downsides. The good thing is that there is a great resource provided by Twitter itself with a very comprehensive [tutorial](https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research). This notebook contains a small extract from it. So if you feel you need more information or something is unclear I would recommend seeing the tutorial.

## What is Twitter?

I would assume that most of you know what Twitter is, what Tweets look like, and what kind of interactions are possible there. Just in the case below there is a very brief definition from the above-mentioned course.

> Twitter is a platform that is used by people across the world to exchange thoughts, ideas, and information with one another, using Tweets. Each Tweet consists of up to 280 characters and may include media such as links, images, and videos. In the context of research, Twitter data refers to the public information that is provided via Twitter’s application programming interface (API). The API supports various endpoints such as recent search, filtered steam, etc. that let developers and researchers connect to the API and request Twitter data.

In general, if you ever saw a tweet. You probably saw it in the format in the picture below.

<center><img src="png/greta.png"/></center>

However, under the hood, there is much more information (metadata) that characterizes every single tweet. With luck we can get some of this information using Twitter API, for example:

 * Tweet text
 * Tweet ID (that uniquely identifies a Tweet)
 * The time at which the Tweet was created
 * Public metrics associated with the Tweet such as the number of retweets, number of likes, etc.
 * Public user information such as username, user ID, user bio, profile image URL, etc.
 * Tweet Annotations - some Tweets are annotated based on the topic that they are about and the named entities present in the Tweets, i.e. COVID-19 stream.

Complete information on the data we can get from each tweet might be found in the documentation under the following [link](https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet).

## Developer Account

As I mentioned in the beginning, to access Twitter API we need to establish a [Developer Account](https://developer.twitter.com/en) (it requires having a regular account first). It requires answering a few fairly easy questions and confirming your email. By default, you will create a Developer Account with Essential Access to Twitter API. For the purpose of this class, it will be enough but if you plan on doing real research it is worth applying for Academic Research Access. 

After creating an account you should be able to log into [Developer Platform](https://developer.twitter.com/en) and create a project. In general, projects serve for organizing your access to Twitter API. Each project might contain multiple Apps (in our case it will be only one app) that serve for generating credentials for authentication. In other words, you need to create a project and later an app so Twitter can recognize that it is you who try to access the API. Therefore, you are never anonymous when you are getting data from Twitter. More or less, they know what you are doing. [Let that sink in](https://edition.cnn.com/videos/business/2022/10/28/late-night-elon-musk-sink-pun-twitter-orig-cprog-fj.cnn-business).

When creating the App and project you simply need to answer a few simple questions. The answer should be straightforward. Until you get to the screen looking like that.

<center><img src="png/keys.png" /></center>

 It is the most important moment because those are the credentials (authorization details you need to store somewhere safe). In this particular case, although it is not the best idea ever just copy and paste them in the following chunk under relevant names. You should, however, try not to share them with anyone. That is because as I mentioned before they serve to identify you (it is more or less your ID for Twitter). So if someone maluses them it will be on your account. You should never share them on public repositories. Probably the best practice is to add them as environmental variables but it is far beyond this class. Therefore, for now, you will store them in this notebook (you can always access them on the Twitter Developer Platform).

In [ ]:
## Define authorization keys
API_KEY = ''
API_SECRET_KEY = ''
BEARER_TOKEN = ''
## For the extraction of the enviornmental variable
import os
BEARER_TOKEN = os.getenv('Bearer_Token')


The just-established account gives us access to the so-called standard product track. In general, it allows to:

* Search for Tweets from the last 7 days by specifying queries using supported operators (more on building queries in later sections)
* Stream Tweets in real-time as they are happening by specifying rules to filter for Tweets that you are interested in.
* Get Tweets from a user’s timeline (up to 3200 most recent Tweets)
* Build the full Tweet objects from a Tweet ID or a set of Tweet IDs
* Look up follower relationships

As you can see it has the limitation of how far we can move back. Moreover, there is a total limit of 500,000 tweets we can get over the month. This restriction, however, does not apply to streaming tweets. We can stream as many tweets as we want. But it is important to acknowledge that the Stream Tweets endpoint gives access to only a sample of tweets (around 1% of all tweets).

### Academic Research Product Track

It is possible and rather straightforward to get access to the whole archive of tweets, dating back to 2006 (using the full-archive search endpoint). It requires just to apply for it and the benefits include:

* Ability to get historical Tweets from the entire archive of public conversation on Twitter, dating back to 2006 (using the full-archive search endpoint)
* Higher monthly Tweet volume cap of 10 million Tweets per month
* More advanced filter options to return relevant data, including a longer query length, support for more concurrent rules (for filtered stream endpoint), and additional operators that are only supported in this product track (more on this later)

As Twitter states: "The Academic Research product track is reserved for those conducting professional academic researchers who have a specific research purpose with Twitter data." In order to get access to the academic research product track, these are the requirements:

* You are a graduate student, doctoral candidate, post-doc, faculty, or research-focused employee at an academic institution or university.
* You have a clearly defined research objective, and you have specific plans for how you intend to use, analyze, and share Twitter data from your research.
* You will use this product track for non-commercial purposes.

From what I know it is not very hard to get such access to Twitter API, however, it requires writing quite a few sentences about the purpose.

## Endpoints

The Twitter API provides different endpoints to get Tweets, based on your use case. It is important to know which endpoint you should use, in order to get the right data. For example, if you want to get historical Tweets, you have the choice of using the [recent search endpoint](https://developer.twitter.com/en/docs/twitter-api/tweets/search/introduction) (if the Tweets are from the last 7 days) or the [full-archive search endpoint](https://developer.twitter.com/en/docs/twitter-api/tweets/search/quick-start/full-archive-search) (if the Tweets are older than that). You can not get this historical data using a streaming endpoint such as [filtered stream endpoint](https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/introduction), because that endpoint only provides Tweets in real-time, as they happen. Similarly, if you want to build your Tweet dataset from a list of Tweet IDs, you can use the [Tweet lookup endpoint](https://developer.twitter.com/en/docs/twitter-api/tweets/lookup/introduction). A good summary of the most popular endpoints with the questions they might help to answer might be found [here](https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/3-deciding-which-endpoints-to-use.md).

In [ ]:
## Load modules
from twarc import Twarc2, expansions ## Twitter Wrapper
import datetime ## Date management
import json ## JSON

## Replace your bearer token below
client = Twarc2(bearer_token=BEARER_TOKEN)


## Specify the start time in UTC for the time period you want Tweets from
## It must be within last 7 days
start_time = datetime.datetime(2022, 11, 1, 0, 0, 0, 0, datetime.timezone.utc)

## Specify the end time in UTC for the time period you want Tweets from
end_time = datetime.datetime(2022, 11, 2, 0, 0, 0, 0, datetime.timezone.utc)

## This is where we specify our query 
query = "from:elonmusk -is:retweet"

## The search_recent method call the recent search endpoint to get Tweets based on the query, start and end times
search_results = client.search_recent(query=query, start_time=start_time, end_time=end_time, max_results=100)

Let's unpack what we got. We accessed the Twitter API using this module `Twarc2` and probably we would expect any kind of information that we succeed. Status code? At least something similar to Wikipedia's response, right?

In [ ]:
## We would expect a status code or a at least a list of JSONs?
search_results

Unfortunately, it is neither. It is a generator object (as it says above). It means that it is the output of a special kind of function that is called a generator. In nutshell, a generator is a function that returns a lazy iterator. It means that its value is generated when it is iterated over. It might be confusing (and for me still is) but we don't have to worry about it right now too much. That is because the good news is that we can use a `for-loop` to extract the values of a generator object.

In [ ]:
## Assign elements of the generator to the 
## object calle results
results = [ item for item in search_results ]

On the other hand a bad news is that since generator generates its content when it is interated over we can do it only once. Therefore, you can only iterate over them once -- they are single use.

In [ ]:
## Let's try to re-use our generator to extract its
## values once again.
[ item for item in search_results ]

As I mentioned before, we should not be very surprised that we got an empty list knowing what a generator is. Good, we assigned the values of the generator to the name `results`. Let's examine it.

In [ ]:
## Let's see what we got from Twitter
print(type(results))
print(len(results))
results

In [ ]:
results[0].keys()

In [ ]:
results[0]['meta']

It is a list of a length 1. And as we see under the index 0 we have a JSON that contains four keys. While we probably will be interested in the `data` others might also draw our attention, for example `meta` and `includes`. They both contain some additional information about tweets (`includes` -- information about media attached to the tweet and `meta` -- simply meta information about tweets we gathered). Therefore, both fields might be of some interest to us. We could try to add them to the relevant tweets in the `data` field but fortunately, we don't have to do it manually. We can use the `expansions.flatten()` function. It will do the dirty work for us.

In [ ]:
results = expansions.flatten(results[0])

Now that we know the structure that we get from Twitter API, we can do all the above-mentioned steps in a single line using a list comprehension.

In [ ]:
## Let's do everything in one line
results_lc = [ tweet  for item in search_results for tweet in expansions.flatten(item) ]
results_lc

Yyyyy, why it did not work? It did not work because we tried to iterate over `search_results` which is a generator object. Therefore, they are only single-iterable.

Ok, let's now see what kind of data we get when we examine a single tweet. It will be probably not a big surprise if I tell you that it is a dictionary. Therefore, let's look at its keys.

In [ ]:
results[0].keys()

In [ ]:
results[0]['in_reply_to_user']

Let's unpack them a bit the fields and recognize what we can expect them to contain.

* `source` (string) -- it indicates the software used to create a tweet, i.e. Twitter for iPhone
* `conversation_id` (string) -- the Tweet ID of the original Tweet of the conversation (which includes direct replies, replies of replies).
* `possibly sensitive` (boolean) -- this field indicates content may be recognized as sensitive. The Tweet author can select within their own account preferences and choose “Mark media you tweet as having material that may be sensitive” so each Tweet created after has this flag set. This may also be judged and labeled by an internal Twitter support agent.
* `reply_settings` (string) -- shows you who can reply to a given Tweet. Fields returned are "everyone", "mentioned_users", and "followers".
* `context_annotations` (list) -- contains context annotations for the Tweet. This is a bit of magic.
* `lang` (string) -- Language of the Tweet, if detected by Twitter.
* `edit_history_tweet_ids` (list) -- unique identifiers indicating all versions of a Tweet. For Tweets with no edits, there will be one ID. For Tweets with an edit history, there will be multiple IDs, arranged in ascending order reflecting the order of edits. The most recent version is the last position of the array.
* `id` (string) -- the unique identifier of the requested Tweet.
* `entities` (dict) -- entities that have been parsed out of the text of the tweet.
* `text` (string) -- the content of the tweet (encoded in UTF-8).
* `edit_controls` (dict) -- when present, this indicates how much longer the Tweet can be edited and the number of remaining edits. Tweets are only editable for the first 30 minutes after creation and can be edited up to five times.
* `in_reply_to_user_id` (string) -- if the represented tweet is a reply, this field will contain the original Tweet’s author ID. This will not necessarily always be the user directly mentioned in the Tweet.
* `created_at` (string) -- creation time of the Tweet (in the following format YYYY-MM-DDThh:mm:ss.<time>Z, i.e. 2022-11-01T23:30:51.000Z).
* `public_metrics` (dict) --  public engagement metrics for the Tweet at the time of the request (retweet count, like count, quote count, reply count).
* `referenced_tweets` (list) --  list of Tweets this Tweet refers to. For example, if the parent Tweet is a Retweet, a Retweet with a comment (also known as Quoted Tweet) or a Reply, it will include the related Tweet referenced to by its parent.
* `author_id` (string) -- the unique identifier of the User who posted this tweet.
* `author` (dict) -- information about the author of the tweet.
* `in_reply_to_user` -- information about the user to which this tweet replies.

### Exercise

Now, that we know what tweets look like. Let's examine the tweets we collected. Please extract from the collected tweets a tweet with the biggest number of likes and return its text (maybe it will be something funny...).

In [ ]:
## YOUR CODE

### Queries

In order to get Tweets for your research using the Twitter API, you need to specify what Tweets you are looking for. To do so, you need to write a search query (when using search endpoints) or set rules (when using the filtered stream endpoint). Both the search query and rules serve the same purpose of giving you the ability to describe the keywords and conditions for which you want tweets.

A search query or rule can consist of a combination of standalone operators such as keywords and conjunction-required operators such as `is:retweet`. Standalone operators can be used by themselves or with other operators, whereas conjunction-related operators can not be used alone.

Below are some examples of search queries based on certain use-cases:

* `from:elonmusk` -- gives all tweets from the Elon Musk account.
* `Elon Musk` -- gives all tweets that contain the words `"Elon Musk"`. The space between the terms indicates logical AND operator. Therefore, both of the words must appear in the tweet.
* `Elon Musk OR unions` -- gives all tweets that contain the words Elon Musk or unions. The logical AND between `Elon` and `Musk` will be applied first followed by the logical OR of that with `union`.
* `from:TwitterDev -is:retweet` -- gives all tweets from the TwitterDev account that are not retweets. The is:retweet operator filters for only those tweets that are retweets and the ‘-’ indicates negation of this condition.
* `covid-19 has:geo` -- gives all Tweets that contain the word covid-19 and have geo data associated with them. However, it only works for the Academic Research product track.
* `from:elonmusk has:images` -- gives all tweets that are from Twitterdev and have images.
* `conversation_id:1394699198382043136` -- gives all tweets in the conversation thread with conversation_id 1394699198382043136. The conversation ID is the tweet ID of the main Tweet for which you want all the replies. Because this Tweet is older than the last 30 days, you can only obtain it using the full-archive search endpoint. Additionally, you will also have to specify the start_time parameter otherwise you will not get any results back.
* `covid-19 lang:pl` -- gives all tweets that contain the word covid-19 that are in Polish.

When you use the recent search and full-archive endpoints as part of the Academic Research product track, your query length can be up to 1024 characters. In the Standard product track, you have access to the recent search endpoint and your query length can be up to 512 characters.

When you use the filtered stream endpoint as part of the Academic Research product track , you can set up to 1000 concurrent rules and each rule can be 1024 characters long. In the Standard product track, you can set 25 concurrent rules and each rule is 512 characters long.

See the complete list of operators supported by endpoints:

1. [Recent search and Full-archive search](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#availability)
2. [Filtered stream](https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/integrate/build-a-rule#availability)

There is also bunch of operators that are exclusive for the Acadmic Research track product but we are going to skip them for now. Their description might be found for example [here](https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/5-how-to-write-search-queries.md).

### Getting a random 1% sample of Tweets in real-time

In [ ]:
## Load modules
from twarc import Twarc2, expansions ## Twitter Wrapper
import json ## JSON

client = Twarc2(bearer_token=BEARER_TOKEN)

list_tweets = []
# The sample method gives a 1% random sample of all Tweets
for count, result in enumerate(client.sample()):
    # The Twitter API v2 returns the Tweet information and the user, media etc.  separately
    # so we use expansions.flatten to get all the information in a single JSON
    tweet = expansions.flatten(result)
    # Here we are printing the full Tweet object JSON to the console
    list_tweets.append(tweet)

    # Replace with the desired number of Tweets, otherwise it will stop after 100 Tweets
    if count > 100:
        break


In [ ]:
list_tweets[101][0].keys()

### Filtering for Tweets on a topic using filtered-stream

The filtered stream endpoints deliver filtered Tweets to you in real-time that match on a set of rules that are applied to the stream. Rules are made up of operators that are used to match on a variety of Tweet attributes.
Multiple rules can be applied to a stream using the method `Twarc2.add_stream_rules()`. Once you’ve added rules and connect to your stream using the `stream` endpoint, only those tweets that match your rules will be delivered in real-time through a persistent streaming connection. You do not need to disconnect from your stream to add or remove rules. 

More details on defining rules might be found [here](https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/integrate/build-a-rule).

In [ ]:
# Load modules
from twarc import Twarc2, expansions ## Twitter Wrapper
import json ## JSON

client = Twarc2(bearer_token=BEARER_TOKEN)

# Remove existing rules
existing_rules = client.get_stream_rules()
if 'data' in existing_rules and len(existing_rules['data']) > 0:
    rule_ids = [rule['id'] for rule in existing_rules['data']]
    client.delete_stream_rule_ids(rule_ids)

# Add new rules
# Replace the rules below with the ones that you want to add as discussed in module 5
new_rules = [
    {"value": "cat has:media", "tag": "cats with media"}
]
added_rules = client.add_stream_rules(rules=new_rules)

list_tweets = []
# Connect to the filtered stream
for count, result in enumerate(client.stream()):
    # The Twitter API v2 returns the Tweet information and the user, media etc.  separately
    # so we use expansions.flatten to get all the information in a single JSON
    tweet = expansions.flatten(result)
    # Here we are printing the full Tweet object JSON to the console
    list_tweets.append(tweet[0])
    # Replace with the desired number of Tweets
    if count > 100:
        break

# Delete the rules once you have collected the desired number of Tweets
rule_ids = [rule['id'] for rule in added_rules['data']]
client.delete_stream_rule_ids(rule_ids)

### Exercise

Write to a JSON file only tweets which author has more than 100 followers.